In [42]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load Dataset from kaggle
dataset = pd.read_csv('..\\Data\\Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [44]:
# select independent and dependent variables
x = dataset.drop(labels= ['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis= 1)
y = dataset['Exited']
# encoding categorical values
from sklearn.preprocessing import LabelEncoder
x['Geography'] = LabelEncoder().fit_transform(x['Geography'])
x['Gender'] = LabelEncoder().fit_transform(x['Gender'])

#avoid dummy variable trapes
x = pd.get_dummies(x, drop_first=True, columns=['Geography'])

In [45]:
x.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,False,False
1,608,0,41,1,83807.86,1,0,1,112542.58,False,True
2,502,0,42,8,159660.80,3,1,0,113931.57,False,False
3,699,0,39,1,0.00,2,0,0,93826.63,False,False
4,850,0,43,2,125510.82,1,1,1,79084.10,False,True


In [46]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [47]:
# SPlitting the dataset  into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=0.2, random_state=0)

# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [48]:
# Build Model
model = tf.keras.models.Sequential()
# Adding layers
# units = 6
# activation function = ReLu
# input dimensions = 11

model.add(tf.keras.layers.Dense(units=6, activation='relu', input_dim = 11))
# Adding second hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu',))
# Adding output layer
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Compile model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = 'accuracy')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 6)                 72        
                                                                 
 dense_8 (Dense)             (None, 6)                 42        
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Train sequential model
model.fit(x_train, y_train.to_numpy(), batch_size=10, epochs=20)  
"""
epochs=20
complete iteration through the entire training dataset. During each epoch, the model is exposed to every training example once, and it updates its weights and biases to learn from the data
"""

Epoch 1/20


800/800 [==============================] - 2s 2ms/step - loss: 0.4774 - accuracy: 0.7960
Epoch 2/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4412 - accuracy: 0.7960
Epoch 3/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4317 - accuracy: 0.7977
Epoch 4/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4255 - accuracy: 0.8133
Epoch 5/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4186 - accuracy: 0.8241
Epoch 6/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4106 - accuracy: 0.8311
Epoch 7/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4034 - accuracy: 0.8331
Epoch 8/20
800/800 [==============================] - 1s 1ms/step - loss: 0.3968 - accuracy: 0.8344
Epoch 9/20
800/800 [==============================] - 1s 1ms/step - loss: 0.3908 - accuracy: 0.8384
Epoch 10/20
800/800 [==============================] - 1s 1ms/step - loss: 0.3862 - accuracy: 0.8379
Epoch 11/2

In [58]:
# model evaluation and prediction
# test_loss , test_acc = model.evaluate(x_test, y_test.to_numpy())

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

63/63 [==============================] - 0s 1ms/step - loss: 0.3551 - accuracy: 0.8505
Test Loss: 0.3551
Test Accuracy: 85.05%


In [60]:
y_pred = model.predict(x_test)

63/63 [==============================] - 0s 1ms/step


In [62]:
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes

In [63]:
y_pred_classes

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [64]:
# Calculate the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[1520   75]
 [ 224  181]]
